In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
gdi = pd.read_csv('../data/curated_gene_disease_associations.tsv', sep = '\t')

In [3]:
gdi.head(2)

,geneId,geneSymbol,DSI,DPI,diseaseId,diseaseName,diseaseType,diseaseClass,diseaseSemanticType,score,EI,YearInitial,YearFinal,NofPmids,NofSnps,source
0,1,A1BG,0.857,0.172,C0019209,Hepatomegaly,phenotype,C06;C23,Finding,0.3,NaN,2017.0,2017.0,1,0,CTD_human
1,1,A1BG,0.857,0.172,C0036341,Schizophrenia,disease,F03,Mental or Behavioral Dysfunction,0.3,NaN,2015.0,2015.0,1,0,CTD_human


In [4]:
gdi = gdi[['geneId','diseaseId']]

In [12]:
gene = list(set(gdi['geneId'].tolist()))
len(set(gdi['geneId'].tolist()))

9413

In [11]:
disease = list(set(gdi['diseaseId'].tolist()))
len(set(gdi['diseaseId'].tolist()))

10370

In [10]:
pd.DataFrame(list(set(gdi['geneId'].tolist() + gdi['diseaseId'].tolist()))).rename(columns={0: "Entity_ID"}).to_csv('../../../../scratch/kh2383/IGCN/data/GDI/entity_list.csv')

In [7]:
entity_list = pd.DataFrame(list(set(gdi['geneId'].tolist() + gdi['diseaseId'].tolist()))).rename(columns={0: "Entity_ID"})

In [95]:
len(entity_list)

19783

In [8]:
len(gdi)

81746

In [14]:
import random

In [9]:
from itertools import product 
# this is too large a number to sample from., we use the alternative method below
comb = list(product(gene, disease))

In [28]:
comb = []
for i in gene:
    comb = comb + (list(zip([i] * 20, random.choices(disease, k = 20))))
    
for j in disease:
    comb = comb + (list(zip(random.choices(gene, k = 20), [i] * 20)))


In [29]:
len(comb)

395660

In [30]:
len(set(comb))

197511

In [10]:
gdi.values

array([[1, 'C0019209'],
       [1, 'C0036341'],
       [2, 'C0002395'],
       ..., 
       [109580095, 'C0019025'],
       [109580095, 'C0085578'],
       [109580095, 'C0271979']], dtype=object)

In [32]:
pos = [(i[0], i[1]) for i in (gdi.values)]

In [33]:
neg = list(set(comb) - set(pos))

In [34]:
len(neg)

197350

In [35]:
comb_flipped = [(i[1], i[0]) for i in comb]

In [36]:
neg_2 = list(set(comb_flipped) - set(pos))

In [38]:
len(neg)

197350

In [39]:
neg_final = list(set(neg) & set(neg_2))

In [40]:
len(neg_final)

197350

In [41]:
len(set(comb))

197511

In [42]:
random.seed(a = 1)
neg_sample = random.sample(neg_final, len(gdi))

In [43]:
len(neg_sample)

81746

In [44]:
df = pd.DataFrame(pos+neg_sample)
df['label'] = np.array([1]*len(pos) + [0]*len(neg_sample))

In [45]:
df = df.rename({0:'Gene_ID', 1:'Disease_ID'}, axis = 1)

In [52]:
def create_fold(df, x):
    test = df.sample(frac = 0.2, replace = False, random_state = x)
    train_val = df[~df.index.isin(test.index)]
    val = train_val.sample(frac = 0.125, replace = False, random_state = 1)
    train = train_val[~train_val.index.isin(val.index)]
    
    path = '/scratch/kh2383/IGCN/data/GDI/fold'+str(x)
    train.reset_index(drop = True).to_csv(path + '/train.csv')
    val.reset_index(drop = True).to_csv(path + '/val.csv')
    test.reset_index(drop = True).to_csv(path + '/test.csv')
    train_val.reset_index(drop = True).to_csv(path + '/train_val.csv')
    
    return train, val, test

In [102]:
for i in range(1, 6):
    train, val, test = create_fold(df, i)